# Healthy Streets of Los Angeles Injuries/Deaths by streets and intersections data project
This project defines streets and streets intersections of Los Angeles with most amount of car accidents resulted in deaths/injuries.

Sources:
* Injuries/Deaths in the City of LA https://tims.berkeley.edu/

Assumptions:
* Location only City of Los Angeles
* Available data is from 2012-2022, note that according to the source data for 2021-2022 is provisional and subject to change
* Excluding freeways (STATE_ROUTE is Null)
* Matching Primary street and Secondary streets if at intersection
* Intersection includes a street if the distance for accident is less then 10 feet for this street

Output:
* Top 500 streets by injuries/deaths with consolidated streets for last 5/10yrs/all time
* Top 100 intersections by injuries/deaths for last 5/10yrs</br>
"last_n years" mean including previous year, but not current one


Any questions - elena.sunchugasheva@gmail.com

In [5]:
# define top amount
top_n_streets = 500
top_n_x = 100

In [6]:
import pandas as pd
import datetime

pd.set_option('display.max_rows', 10000)
pd.set_option('display.max_columns', 1000)

# data

In [7]:
today = datetime.datetime.now()
print(today)

2023-12-12 19:47:12.376722


### Data preparation

In [8]:
crashes_raw = pd.read_csv('../Crashes.csv')
display(crashes_raw.head(1))

,CASE_ID,ACCIDENT_YEAR,PROC_DATE,JURIS,COLLISION_DATE,COLLISION_TIME,OFFICER_ID,REPORTING_DISTRICT,DAY_OF_WEEK,CHP_SHIFT,POPULATION,CNTY_CITY_LOC,SPECIAL_COND,BEAT_TYPE,CHP_BEAT_TYPE,CITY_DIVISION_LAPD,CHP_BEAT_CLASS,BEAT_NUMBER,PRIMARY_RD,SECONDARY_RD,DISTANCE,DIRECTION,INTERSECTION,WEATHER_1,WEATHER_2,STATE_HWY_IND,CALTRANS_COUNTY,CALTRANS_DISTRICT,STATE_ROUTE,ROUTE_SUFFIX,POSTMILE_PREFIX,POSTMILE,LOCATION_TYPE,RAMP_INTERSECTION,SIDE_OF_HWY,TOW_AWAY,COLLISION_SEVERITY,NUMBER_KILLED,NUMBER_INJURED,PARTY_COUNT,PRIMARY_COLL_FACTOR,PCF_CODE_OF_VIOL,PCF_VIOL_CATEGORY,PCF_VIOLATION,PCF_VIOL_SUBSECTION,HIT_AND_RUN,TYPE_OF_COLLISION,MVIW,PED_ACTION,ROAD_SURFACE,ROAD_COND_1,ROAD_COND_2,LIGHTING,CONTROL_DEVICE,CHP_ROAD_TYPE,PEDESTRIAN_ACCIDENT,BICYCLE_ACCIDENT,MOTORCYCLE_ACCIDENT,TRUCK_ACCIDENT,NOT_PRIVATE_PROPERTY,ALCOHOL_INVOLVED,STWD_VEHTYPE_AT_FAULT,CHP_VEHTYPE_AT_FAULT,COUNT_SEVERE_INJ,COUNT_VISIBLE_INJ,COUNT_COMPLAINT_PAIN,COUNT_PED_KILLED,COUNT_PED_INJURED,COUNT_BICYCLIST_KILLED,COUNT_BICYCLIST_INJURED,COUNT_MC_KILLED,COUNT_MC_INJURED,PRIMARY_RAMP,SECONDARY_RAMP,LATITUDE,LONGITUDE,COUNTY,CITY,POINT_X,POINT_Y
0,5378005,2011,2012-12-27,1900,2011-09-25,1718,517840,2607,7,5,6,1950,0,0,0,NaN,0,264T1,AVENUE S,55TH ST EAST,0.0,NaN,Y,A,-,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,3,0,4,2,A,-,09,21801.0,A,N,A,C,A,A,H,-,A,A,0,NaN,NaN,NaN,NaN,Y,NaN,A,01,0,1,3,0,0,0,0,0,0,-,-,NaN,NaN,LOS ANGELES,PALMDALE,-118.031586,34.5581


In [9]:
# columns we are interested in
crashes_col = [
    'CASE_ID', 'COUNTY', 'CITY',
    'ACCIDENT_YEAR', 'COLLISION_DATE',
    'PRIMARY_RD', 'SECONDARY_RD', 'POINT_X', 'POINT_Y',
    'INTERSECTION', 'DISTANCE', 
    'COLLISION_SEVERITY', 'NUMBER_KILLED', 'NUMBER_INJURED', 'PARTY_COUNT',
    'PED_ACTION', 'PEDESTRIAN_ACCIDENT', 'BICYCLE_ACCIDENT',
    'COUNT_SEVERE_INJ', 'COUNT_VISIBLE_INJ',
    'COUNT_PED_KILLED', 'COUNT_PED_INJURED',
    'COUNT_BICYCLIST_KILLED', 'COUNT_BICYCLIST_INJURED'
]

In [10]:
# take only City of LA and not highways
crashes = crashes_raw[
        (crashes_raw.CITY=='LOS ANGELES')&
        (crashes_raw.STATE_ROUTE.isnull())
    ][crashes_col].copy()
display(crashes.head(1))

,CASE_ID,COUNTY,CITY,ACCIDENT_YEAR,COLLISION_DATE,PRIMARY_RD,SECONDARY_RD,POINT_X,POINT_Y,INTERSECTION,DISTANCE,COLLISION_SEVERITY,NUMBER_KILLED,NUMBER_INJURED,PARTY_COUNT,PED_ACTION,PEDESTRIAN_ACCIDENT,BICYCLE_ACCIDENT,COUNT_SEVERE_INJ,COUNT_VISIBLE_INJ,COUNT_PED_KILLED,COUNT_PED_INJURED,COUNT_BICYCLIST_KILLED,COUNT_BICYCLIST_INJURED
12,8410058,LOS ANGELES,LOS ANGELES,2017,2017-06-26,92ND ST,COMPTON,-118.24751,33.95334,Y,0.0,4,0,1,2,A,NaN,NaN,0,0,0,0,0,0


take a look at stats/outliers

In [11]:
print(
    'intersections:', crashes[crashes.INTERSECTION=='Y'].shape[0],
    ', non-intersections:', crashes[crashes.INTERSECTION=='N'].shape[0],
    ', total:', crashes.shape[0]
)

intersections: 98002 , non-intersections: 128935 , total: 228033


In [12]:
crashes[[
    'NUMBER_KILLED', 'NUMBER_INJURED',
    'COUNT_PED_KILLED', 'COUNT_PED_INJURED',
    'COUNT_BICYCLIST_KILLED', 'COUNT_BICYCLIST_INJURED'
]].describe()

,NUMBER_KILLED,NUMBER_INJURED,COUNT_PED_KILLED,COUNT_PED_INJURED,COUNT_BICYCLIST_KILLED,COUNT_BICYCLIST_INJURED
count,228033.000000,228033.000000,228033.000000,228033.000000,228033.000000,228033.000000
mean,0.011332,1.472379,0.005249,0.137313,0.000746,0.089158
std,0.111299,0.897703,0.073106,0.370881,0.027772,0.287546
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000
max,4.000000,39.000000,2.000000,14.000000,3.000000,4.000000


In [13]:
crashes[crashes.NUMBER_INJURED>=20]

,CASE_ID,COUNTY,CITY,ACCIDENT_YEAR,COLLISION_DATE,PRIMARY_RD,SECONDARY_RD,POINT_X,POINT_Y,INTERSECTION,DISTANCE,COLLISION_SEVERITY,NUMBER_KILLED,NUMBER_INJURED,PARTY_COUNT,PED_ACTION,PEDESTRIAN_ACCIDENT,BICYCLE_ACCIDENT,COUNT_SEVERE_INJ,COUNT_VISIBLE_INJ,COUNT_PED_KILLED,COUNT_PED_INJURED,COUNT_BICYCLIST_KILLED,COUNT_BICYCLIST_INJURED
26781,6364042,LOS ANGELES,LOS ANGELES,2014,2014-01-10,VERNON AV,HALLDALE AV,-118.302360,34.003710,Y,0.0,3,0,22,2,A,NaN,NaN,0,4,0,0,0,0
216271,5760917,LOS ANGELES,LOS ANGELES,2012,2012-08-27,EAST WASHINGTON BL,S SAN PEDRO ST,-118.257398,34.027678,N,5.0,2,0,39,3,A,NaN,NaN,1,4,0,0,0,0
352262,5830650,LOS ANGELES,LOS ANGELES,2012,2012-10-23,STANLEY AV,HOLLYWOOD BL,-118.356212,34.101675,Y,0.0,3,0,34,4,A,NaN,NaN,0,11,0,0,0,0
374527,8008423,LOS ANGELES,LOS ANGELES,2016,2016-07-11,VICKSBURG,SEPULVEDA BL,NaN,NaN,N,1000.0,3,0,20,2,A,NaN,NaN,0,1,0,0,0,0
449347,8129625,LOS ANGELES,LOS ANGELES,2016,2016-09-06,SUNLAND BL,SAN FERNANDO RD,-118.368865,34.219262,N,67.0,4,0,20,3,A,NaN,NaN,0,0,0,0,0,0


# stats

## functions

In [14]:
def join_streets(col1, col2):
    return '/'.join(sorted([str(col1), str(col2)]))

In [15]:
def summarize(df, summ_column, time_period=None, today=today):
    '''
    df - DataFrame with the stucture of crashes_street
    summ_column - summarize on column (excluding 'county', 'city')
    time_period = ('last5', 'last10') - time filter
    '''
    if time_period:   
        df = df[
            df.ACCIDENT_YEAR.isin(
                range(
                    today.year-int(time_period[4:]),
                    today.year
                )
            )
        ]
    
    df_return = df\
        .sort_values(by=['COUNTY', 'CITY', summ_column])\
        .groupby(by=['COUNTY', 'CITY', summ_column]).sum()\
        .reset_index(drop=False)\
        .drop(columns='ACCIDENT_YEAR')
    
    df_return.columns = [
        'COUNTY', 'CITY', summ_column
        ] + list(df_return.columns[3:] + '_' + time_period)

    return df_return

In [16]:
def get_top_n(df, top_n, summ_column, time_period, outcome):
    '''
    df - DataFrame with the stucture of crashes_street
    summ_column - summarize on column
    time_period = ('last5', 'last10') - time filter
    outcome = ('killed', 'injured') - outcome of the crash
    '''
    df_cols = list(df.columns[df.columns.str.contains(time_period)])
    sort_cols = [col for col in df_cols if 'number' in col.lower()]
    first = [col for col in sort_cols if outcome in col.lower()][0]
    sort_cols.remove(first)
    second = sort_cols[0]
    if len(sort_cols)!=1:
        second = sorted(second)[0]

    df_top = df[[summ_column] + df_cols].copy()\
        .sort_values(
            by=[first, second],
            ascending=False
        ).head(top_n)
    
    return df_top

## top streets

### prepare dataset

In [17]:
# if the crash happened at intesection - count secondary street as well
crashes_secondary = crashes[
        crashes.INTERSECTION=='Y'
    ].copy().reset_index(drop=True)
crashes_secondary.PRIMARY_RD = crashes_secondary.SECONDARY_RD
crashes_primary_secondary = pd.concat([crashes, crashes_secondary])
display(crashes_secondary.tail(3))
display(crashes_primary_secondary.tail(3))

,CASE_ID,COUNTY,CITY,ACCIDENT_YEAR,COLLISION_DATE,PRIMARY_RD,SECONDARY_RD,POINT_X,POINT_Y,INTERSECTION,DISTANCE,COLLISION_SEVERITY,NUMBER_KILLED,NUMBER_INJURED,PARTY_COUNT,PED_ACTION,PEDESTRIAN_ACCIDENT,BICYCLE_ACCIDENT,COUNT_SEVERE_INJ,COUNT_VISIBLE_INJ,COUNT_PED_KILLED,COUNT_PED_INJURED,COUNT_BICYCLIST_KILLED,COUNT_BICYCLIST_INJURED
97999,9519872,LOS ANGELES,LOS ANGELES,2022,2022-12-31,FIGUEROA ST,FIGUEROA ST,-118.282722,33.950089,Y,0.0,3,0,1,2,A,NaN,NaN,0,1,0,0,0,0
98000,9566696,LOS ANGELES,LOS ANGELES,2022,2022-03-06,JAMES M WOOD BL,JAMES M WOOD BL,-118.291634,34.055939,Y,0.0,4,0,1,2,A,NaN,NaN,0,0,0,0,0,0
98001,9571054,LOS ANGELES,LOS ANGELES,2022,2022-10-23,PACKARD ST,PACKARD ST,-118.359238,34.052067,Y,0.0,3,0,1,2,A,NaN,NaN,0,1,0,0,0,0


,CASE_ID,COUNTY,CITY,ACCIDENT_YEAR,COLLISION_DATE,PRIMARY_RD,SECONDARY_RD,POINT_X,POINT_Y,INTERSECTION,DISTANCE,COLLISION_SEVERITY,NUMBER_KILLED,NUMBER_INJURED,PARTY_COUNT,PED_ACTION,PEDESTRIAN_ACCIDENT,BICYCLE_ACCIDENT,COUNT_SEVERE_INJ,COUNT_VISIBLE_INJ,COUNT_PED_KILLED,COUNT_PED_INJURED,COUNT_BICYCLIST_KILLED,COUNT_BICYCLIST_INJURED
97999,9519872,LOS ANGELES,LOS ANGELES,2022,2022-12-31,FIGUEROA ST,FIGUEROA ST,-118.282722,33.950089,Y,0.0,3,0,1,2,A,NaN,NaN,0,1,0,0,0,0
98000,9566696,LOS ANGELES,LOS ANGELES,2022,2022-03-06,JAMES M WOOD BL,JAMES M WOOD BL,-118.291634,34.055939,Y,0.0,4,0,1,2,A,NaN,NaN,0,0,0,0,0,0
98001,9571054,LOS ANGELES,LOS ANGELES,2022,2022-10-23,PACKARD ST,PACKARD ST,-118.359238,34.052067,Y,0.0,3,0,1,2,A,NaN,NaN,0,1,0,0,0,0


In [18]:
# check duplicates
print(crashes_primary_secondary.shape[0])
crashes_primary_secondary.drop_duplicates(inplace=True)
print(crashes_primary_secondary.shape[0])

326035
326035


### get stats

In [19]:
# get main dataset for stats
crashes_street = crashes[[
    'COUNTY', 'CITY', 'ACCIDENT_YEAR', 'PRIMARY_RD',
    'NUMBER_KILLED', 'NUMBER_INJURED', 'COUNT_PED_KILLED', 'COUNT_PED_INJURED',
    'COUNT_BICYCLIST_KILLED', 'COUNT_BICYCLIST_INJURED'
]].groupby(by=['COUNTY', 'CITY', 'ACCIDENT_YEAR', 'PRIMARY_RD']).sum()\
.reset_index(drop=False)
display(crashes_street.head(2))

,COUNTY,CITY,ACCIDENT_YEAR,PRIMARY_RD,NUMBER_KILLED,NUMBER_INJURED,COUNT_PED_KILLED,COUNT_PED_INJURED,COUNT_BICYCLIST_KILLED,COUNT_BICYCLIST_INJURED
0,LOS ANGELES,LOS ANGELES,2011,100TH AV,0,3,0,0,0,0
1,LOS ANGELES,LOS ANGELES,2011,101ST ST,0,5,0,2,0,0


In [20]:
# get the list of all streets and fill it in
crashes_street_stat = crashes_primary_secondary[[
    'COUNTY', 'CITY', 'PRIMARY_RD'
]].copy().drop_duplicates()

intervals = ['last5', 'last10']

for interval in intervals:
    df_summary = summarize(
        df = crashes_street,
        summ_column = 'PRIMARY_RD',
        time_period = interval
    )
    crashes_street_stat = crashes_street_stat.merge(
        df_summary,
        how='left',
        on=['COUNTY', 'CITY', 'PRIMARY_RD']
    )
    
display(crashes_street_stat.head(2))

,COUNTY,CITY,PRIMARY_RD,NUMBER_KILLED_last5,NUMBER_INJURED_last5,COUNT_PED_KILLED_last5,COUNT_PED_INJURED_last5,COUNT_BICYCLIST_KILLED_last5,COUNT_BICYCLIST_INJURED_last5,NUMBER_KILLED_last10,NUMBER_INJURED_last10,COUNT_PED_KILLED_last10,COUNT_PED_INJURED_last10,COUNT_BICYCLIST_KILLED_last10,COUNT_BICYCLIST_INJURED_last10
0,LOS ANGELES,LOS ANGELES,92ND ST,2.0,140.0,1.0,10.0,0.0,10.0,3.0,332.0,1.0,24.0,0.0,25.0
1,LOS ANGELES,LOS ANGELES,VERMONT AV,13.0,1247.0,8.0,123.0,1.0,66.0,30.0,3742.0,21.0,355.0,1.0,280.0


In [21]:
# find top = top_n_streets streets on killed/injures and make one list
top_streets = set()
outcome = ['killed', 'injured']

for interval in intervals:
    for out in outcome:
        print(interval, 'years,', out)
        top_n = get_top_n(
            df = crashes_street_stat,
            top_n = top_n_streets,
            summ_column = 'PRIMARY_RD',
            time_period = interval,
            outcome = out
        )
        top_streets.update(top_n.PRIMARY_RD.unique())
    
top_streets_df = crashes_street_stat[
    crashes_street_stat.PRIMARY_RD.isin(list(top_streets))
    ].reset_index(drop=True)

last5 years, killed
last5 years, injured
last10 years, killed
last10 years, injured


In [22]:
top_streets_df.to_csv(f'top{top_n_streets}_streets_combined_{today.strftime("%Y_%m_%d")}.csv', index=False)

## top intersections

### create intersection column

In [23]:
# these are probably outliers, may be look into it later
crashes[
    (crashes.INTERSECTION=='Y')&(crashes.DISTANCE > 15)
].sort_values(
    by='DISTANCE',
    ascending=False
).shape

(66, 24)

In [24]:
# lets take all crashes within 15 ft distance from intersections
crashes_intersection = crashes[
        crashes.DISTANCE<=15
    ].copy().reset_index(drop=True)

# get an intersection combined name
crashes_intersection['cross_name'] = crashes_intersection.apply(
    lambda row: join_streets(
        row['PRIMARY_RD'], row['SECONDARY_RD']
    ), axis=1
)
display(crashes_intersection.head(2))

,CASE_ID,COUNTY,CITY,ACCIDENT_YEAR,COLLISION_DATE,PRIMARY_RD,SECONDARY_RD,POINT_X,POINT_Y,INTERSECTION,DISTANCE,COLLISION_SEVERITY,NUMBER_KILLED,NUMBER_INJURED,PARTY_COUNT,PED_ACTION,PEDESTRIAN_ACCIDENT,BICYCLE_ACCIDENT,COUNT_SEVERE_INJ,COUNT_VISIBLE_INJ,COUNT_PED_KILLED,COUNT_PED_INJURED,COUNT_BICYCLIST_KILLED,COUNT_BICYCLIST_INJURED,cross_name
0,8410058,LOS ANGELES,LOS ANGELES,2017,2017-06-26,92ND ST,COMPTON,-118.24751,33.95334,Y,0.0,4,0,1,2,A,NaN,NaN,0,0,0,0,0,0,92ND ST/COMPTON
1,8193899,LOS ANGELES,LOS ANGELES,2016,2016-12-05,VERMONT AV,108TH ST,NaN,NaN,Y,0.0,4,0,3,4,A,NaN,NaN,0,0,0,0,0,0,108TH ST/VERMONT AV


In [25]:
# get average coordinates for intersections from accidents on the intersection=Y
# so we can count it as coordinates for intersection
intersection_coords_list = []

for inters in crashes_intersection.cross_name.unique():
    cross_locations = crashes_intersection[
            (crashes_intersection.cross_name==inters)&
            (crashes_intersection.INTERSECTION=='Y')
        ]
    intersection_coords_list.append({
        'cross_name':inters,
        'cross_x':cross_locations.POINT_X.mean(),
        'cross_y':cross_locations.POINT_Y.mean()
    })

In [26]:
intersection_coords = pd.DataFrame(intersection_coords_list)
display(intersection_coords.head(2))

crashes_intersections = crashes_intersection.merge(intersection_coords, on='cross_name')
display(crashes_intersections.head(2))

,cross_name,cross_x,cross_y
0,92ND ST/COMPTON,-118.247510,33.953340
1,108TH ST/VERMONT AV,-118.291708,33.938195


,CASE_ID,COUNTY,CITY,ACCIDENT_YEAR,COLLISION_DATE,PRIMARY_RD,SECONDARY_RD,POINT_X,POINT_Y,INTERSECTION,DISTANCE,COLLISION_SEVERITY,NUMBER_KILLED,NUMBER_INJURED,PARTY_COUNT,PED_ACTION,PEDESTRIAN_ACCIDENT,BICYCLE_ACCIDENT,COUNT_SEVERE_INJ,COUNT_VISIBLE_INJ,COUNT_PED_KILLED,COUNT_PED_INJURED,COUNT_BICYCLIST_KILLED,COUNT_BICYCLIST_INJURED,cross_name,cross_x,cross_y
0,8410058,LOS ANGELES,LOS ANGELES,2017,2017-06-26,92ND ST,COMPTON,-118.247510,33.953340,Y,0.0,4,0,1,2,A,NaN,NaN,0,0,0,0,0,0,92ND ST/COMPTON,-118.24751,33.95334
1,8941972,LOS ANGELES,LOS ANGELES,2019,2019-09-01,92ND ST,COMPTON,-118.247528,33.953339,N,5.0,4,0,1,2,A,NaN,Y,0,0,0,0,0,1,92ND ST/COMPTON,-118.24751,33.95334


In [27]:
# check intersections with locations far from the averaged value
# they still look not too far
crashes_intersections.loc[
    (crashes_intersections.INTERSECTION=='Y')&
    ((crashes_intersections.POINT_X-crashes_intersections.cross_x>0.1)|
     (crashes_intersections.POINT_Y-crashes_intersections.cross_y>0.1)),
    ['cross_name', 'cross_x', 'cross_y', 'POINT_X', 'POINT_Y']
]

,cross_name,cross_x,cross_y,POINT_X,POINT_Y
1482,10TH ST/PACIFIC AV,-118.316784,33.772757,-118.403320,33.885630
3163,5TH ST/MAIN ST,-118.181725,34.040799,-117.947890,34.020150
3171,5TH ST/MAIN ST,-118.181725,34.040799,-117.947890,34.020150
4955,3RD ST/CATALINA ST,-118.302175,33.907554,-118.295410,34.069050
4956,3RD ST/CATALINA ST,-118.302175,33.907554,-118.295410,34.069050
4957,3RD ST/CATALINA ST,-118.302175,33.907554,-118.295410,34.069050
4959,3RD ST/CATALINA ST,-118.302175,33.907554,-118.295410,34.069050
4961,3RD ST/CATALINA ST,-118.302175,33.907554,-118.295410,34.069050
4966,3RD ST/CATALINA ST,-118.302175,33.907554,-118.295410,34.069050
4968,3RD ST/CATALINA ST,-118.302175,33.907554,-118.295410,34.069050


### get stats

In [28]:
# get a base for the final table
crashes_cross_stat = crashes_intersections[[
    'COUNTY', 'CITY', 'cross_name', 'cross_x', 'cross_y'
]].copy().drop_duplicates()
display(crashes_cross_stat.head())

# get separate streets
crashes_cross_stat['street1'] = ''
crashes_cross_stat['street2'] = ''

crashes_cross_stat.street1 = crashes_cross_stat.cross_name.str.split('/').str[0]
crashes_cross_stat.street2 = crashes_cross_stat.cross_name.str.split('/').str[1]

,COUNTY,CITY,cross_name,cross_x,cross_y
0,LOS ANGELES,LOS ANGELES,92ND ST/COMPTON,-118.247510,33.953340
2,LOS ANGELES,LOS ANGELES,108TH ST/VERMONT AV,-118.291708,33.938195
21,LOS ANGELES,LOS ANGELES,HAZELTINE AV/HUSTON ST,-118.440002,34.159409
27,LOS ANGELES,LOS ANGELES,165TH PL/AINSWORTH AV,NaN,NaN
28,LOS ANGELES,LOS ANGELES,OLYMPIC BL/SAINT ANDREWS PL,-118.311158,34.052638


In [29]:
# get a list of crashes for statistics
crashes_cross = crashes_intersections[[
    'COUNTY', 'CITY', 'ACCIDENT_YEAR', 'cross_name',
    'NUMBER_KILLED', 'NUMBER_INJURED', 'COUNT_PED_KILLED', 'COUNT_PED_INJURED',
    'COUNT_BICYCLIST_KILLED', 'COUNT_BICYCLIST_INJURED'
]].groupby(by=['COUNTY', 'CITY', 'ACCIDENT_YEAR', 'cross_name']).sum()\
.reset_index(drop=False)

intervals = ['last5', 'last10']
for interval in intervals:
    df_summary = summarize(
        df = crashes_cross,
        summ_column = 'cross_name',
        time_period = interval
    )
    crashes_cross_stat = crashes_cross_stat.merge(
        df_summary,
        how='left',
        on=['COUNTY', 'CITY', 'cross_name']
    )
    
display(crashes_cross_stat.head(2))

,COUNTY,CITY,cross_name,cross_x,cross_y,street1,street2,NUMBER_KILLED_last5,NUMBER_INJURED_last5,COUNT_PED_KILLED_last5,COUNT_PED_INJURED_last5,COUNT_BICYCLIST_KILLED_last5,COUNT_BICYCLIST_INJURED_last5,NUMBER_KILLED_last10,NUMBER_INJURED_last10,COUNT_PED_KILLED_last10,COUNT_PED_INJURED_last10,COUNT_BICYCLIST_KILLED_last10,COUNT_BICYCLIST_INJURED_last10
0,LOS ANGELES,LOS ANGELES,92ND ST/COMPTON,-118.247510,33.953340,92ND ST,COMPTON,0.0,1.0,0.0,0.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,1.0
1,LOS ANGELES,LOS ANGELES,108TH ST/VERMONT AV,-118.291708,33.938195,108TH ST,VERMONT AV,0.0,15.0,0.0,0.0,0.0,1.0,0.0,35.0,0.0,0.0,0.0,1.0


In [30]:
# find top = top_n_streets streets on killed/injures and make one list
top_intersections = set()
outcome = ['killed', 'injured']

for interval in intervals:
    for out in outcome:
        print(interval, 'years,', out)
        top_n = get_top_n(
            df = crashes_cross_stat,
            top_n = top_n_x,
            summ_column = 'cross_name',
            time_period = interval,
            outcome = out
        )
        top_intersections.update(top_n.cross_name.unique())
    
top_intersections_df = crashes_cross_stat[
        crashes_cross_stat.cross_name.isin(list(top_intersections))
    ].reset_index(
        drop=True
    ).sort_values(
        by=['NUMBER_KILLED_last5', 'NUMBER_INJURED_last5'],
        ascending=False
    )

last5 years, killed
last5 years, injured
last10 years, killed
last10 years, injured


In [31]:
top_intersections_df.to_csv(f'top{top_n_x}_intersections_combined_{today.strftime("%Y_%m_%d")}.csv', index=False)

### more checks

In [32]:
# some checks
display(top_intersections_df[top_intersections_df.cross_x==0])
display(top_intersections_df[top_intersections_df.street1.isnull()])

,COUNTY,CITY,cross_name,cross_x,cross_y,street1,street2,NUMBER_KILLED_last5,NUMBER_INJURED_last5,COUNT_PED_KILLED_last5,COUNT_PED_INJURED_last5,COUNT_BICYCLIST_KILLED_last5,COUNT_BICYCLIST_INJURED_last5,NUMBER_KILLED_last10,NUMBER_INJURED_last10,COUNT_PED_KILLED_last10,COUNT_PED_INJURED_last10,COUNT_BICYCLIST_KILLED_last10,COUNT_BICYCLIST_INJURED_last10


,COUNTY,CITY,cross_name,cross_x,cross_y,street1,street2,NUMBER_KILLED_last5,NUMBER_INJURED_last5,COUNT_PED_KILLED_last5,COUNT_PED_INJURED_last5,COUNT_BICYCLIST_KILLED_last5,COUNT_BICYCLIST_INJURED_last5,NUMBER_KILLED_last10,NUMBER_INJURED_last10,COUNT_PED_KILLED_last10,COUNT_PED_INJURED_last10,COUNT_BICYCLIST_KILLED_last10,COUNT_BICYCLIST_INJURED_last10


In [33]:
print(
    crashes_intersections[
        crashes_intersections.cross_x==0
    ].cross_name.nunique(),
    crashes_intersections[
        crashes_intersections.cross_x.isnull()
    ].cross_name.nunique(),
)

0 7705


In [34]:
print(
    crashes_intersections[crashes_intersections.cross_x!=0].shape,
    crashes_intersections[crashes_intersections.cross_x==0].shape,
    crashes_intersections[crashes_intersections.cross_x.isnull()].shape,
)

(123512, 27) (0, 27) (8200, 27)


In [35]:
top_intersections_df[top_intersections_df.cross_x.isnull()]

,COUNTY,CITY,cross_name,cross_x,cross_y,street1,street2,NUMBER_KILLED_last5,NUMBER_INJURED_last5,COUNT_PED_KILLED_last5,COUNT_PED_INJURED_last5,COUNT_BICYCLIST_KILLED_last5,COUNT_BICYCLIST_INJURED_last5,NUMBER_KILLED_last10,NUMBER_INJURED_last10,COUNT_PED_KILLED_last10,COUNT_PED_INJURED_last10,COUNT_BICYCLIST_KILLED_last10,COUNT_BICYCLIST_INJURED_last10
231,LOS ANGELES,LOS ANGELES,BEVERLY BL/N IRVING BL,NaN,NaN,BEVERLY BL,N IRVING BL,2.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0
238,LOS ANGELES,LOS ANGELES,62ND ST/SAINT ANDREWS PL,NaN,NaN,62ND ST,SAINT ANDREWS PL,2.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0
233,LOS ANGELES,LOS ANGELES,HASKELL AV/VINTAGE ST,NaN,NaN,HASKELL AV,VINTAGE ST,NaN,NaN,NaN,NaN,NaN,NaN,2.0,1.0,0.0,0.0,0.0,0.0


In [36]:
top_intersections_df[top_intersections_df.cross_name=='HASKELL AV/VINTAGE ST']

,COUNTY,CITY,cross_name,cross_x,cross_y,street1,street2,NUMBER_KILLED_last5,NUMBER_INJURED_last5,COUNT_PED_KILLED_last5,COUNT_PED_INJURED_last5,COUNT_BICYCLIST_KILLED_last5,COUNT_BICYCLIST_INJURED_last5,NUMBER_KILLED_last10,NUMBER_INJURED_last10,COUNT_PED_KILLED_last10,COUNT_PED_INJURED_last10,COUNT_BICYCLIST_KILLED_last10,COUNT_BICYCLIST_INJURED_last10
233,LOS ANGELES,LOS ANGELES,HASKELL AV/VINTAGE ST,NaN,NaN,HASKELL AV,VINTAGE ST,NaN,NaN,NaN,NaN,NaN,NaN,2.0,1.0,0.0,0.0,0.0,0.0
